In [57]:
import numpy as np
import xbbg 
from xbbg import blp
import plotly.express as px
import plotly.graph_objects as go

In [58]:
stock = blp.bdh('AAPL US Equity', ["PX_Last", "PR005", "RT115", "RR902"], start_date="1/1/2016")
print(stock)

           AAPL US Equity                                 
                  PX_Last    PR005     RT115         RR902
2016-01-04         24.042   24.042    0.0000  5.873611e+05
2016-01-05         23.439   23.439   -2.5059  5.726423e+05
2016-01-06         22.980   22.980   -4.4139  5.583298e+05
2016-01-07         22.010   22.010   -8.4480  5.347658e+05
2016-01-08         22.127   22.127   -7.9639  5.375934e+05
...                   ...      ...       ...           ...
2023-06-20        185.010  185.010  669.6434  2.909967e+06
2023-06-21        183.960  183.960  665.2754  2.893452e+06
2023-06-22        187.000  187.000  677.9218  2.941267e+06
2023-06-23        186.680  186.680  676.5906  2.936234e+06
2023-06-26            NaN      NaN       NaN  2.936234e+06

[1882 rows x 4 columns]


In [59]:
px_arr = stock[('AAPL US Equity', 'PX_Last')].to_numpy()
mask = np.isnan(px_arr)
px_arr_ = px_arr[~mask]
date_arr = stock.index.values

In [60]:
def directional_change(prices: np.array, date: np.array, sigma:float= 0.03):
    nextUp = True
    tmp_max = 0
    tmp_min = 0
    tmp_max_i = 0
    tmp_min_i = 0
    x2 = [date[0]]
    y2 = [prices[0]]
    for i in range(len(prices)):
        if nextUp:
            if prices[i]>tmp_max:
                tmp_max = prices[i]
                tmp_max_i = i
            elif prices[i]<tmp_max-tmp_max*sigma:
                x2.append(date[tmp_max_i])
                y2.append(tmp_max)
                nextUp = False
                tmp_min = prices[i]
                tmp_min_i = i

        elif not nextUp:
            if prices[i]<tmp_min:
                tmp_min = prices[i]
                tmp_min_i = i
            elif prices[i]>tmp_min+tmp_min*sigma:
                x2.append(date[tmp_min_i])
                y2.append(tmp_min)
                nextUp = True
                tmp_max = prices[i]
                tmp_max_i = i
    if nextUp:
        x2.append(date[tmp_max_i])
        y2.append(tmp_max)
    else:
        x2.append(date[tmp_min_i])
        y2.append(tmp_min)
    return x2, y2

In [16]:

x2, y2 = directional_change(px_arr_,date_arr, 0.05)

fig = go.Figure()

# add the line trace using add_trace() method
fig.add_trace(go.Scatter(x = date_arr, y=px_arr_, mode='lines', name='Stock price'))

# add the scatter trace using add_trace() method
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', name='Trend Line'))

# customize the layout
fig.update_layout(title='Stock Price With Trend Line',
                  xaxis_title='Date',
                  yaxis_title='Price')

# show the figure
fig.show()

In [17]:
x2, y2 = directional_change(px_arr_,date_arr, 0.19)

fig = go.Figure()

# add the line trace using add_trace() method
fig.add_trace(go.Scatter(x = date_arr, y=px_arr_, mode='lines', name='Stock price'))

# add the scatter trace using add_trace() method
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', name='Trend Line'))

# customize the layout
fig.update_layout(title='Stock Price With Trend Line',
                  xaxis_title='Date',
                  yaxis_title='Price')

# show the figure
fig.show()

In [53]:
def rw_local_max(prices: np.array, index:int, order:int):
    if index < order * 2 + 1:
        return False
    else:
        is_max = True
        k = index - order
        v = prices[k]
        for i in range(1, order+1):
            if prices[k+i]>v or prices[k-i]>v :
                is_max = False
                break
        return is_max
        
    
def rw_local_min(prices: np.array, index:int, order:int):
    if index < order * 2 + 1:
        return False
    else:
        is_min = True
        k = index - order
        v = prices[k]
        for i in range(1, order+1):
            if prices[k+i]<v or prices[k-i]<v :
                is_min = False
                break
        return is_min
def rolling_window (prices: np.array, date: np.array, order:int= 3):
    x2 = [date[0]]
    y2 = [prices[0]]
    for i in range(len(prices)):
        if rw_local_max(prices, i, order):
            x2.append(date[i-order])
            y2.append(prices[i-order])
        elif rw_local_min(prices, i, order):
            x2.append(date[i-order])
            y2.append(prices[i-order])
    x2.append(date[-1])
    y2.append(prices[-1])
    return x2, y2

In [56]:
x2, y2 = rolling_window(px_arr_,date_arr, 15)

fig = go.Figure()

# add the line trace using add_trace() method
fig.add_trace(go.Scatter(x = date_arr, y=px_arr_, mode='lines', name='Stock price'))

# add the scatter trace using add_trace() method
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines', name='Trend Line'))

# customize the layout
fig.update_layout(title='Stock Price With Trend Line',
                  xaxis_title='Date',
                  yaxis_title='Price')

# show the figure
fig.show()